In [9]:
from langchain_community.document_loaders.csv_loader import CSVLoader

file_path = (
    "./linux/Linux_2k.log_templates.csv"
)

# file_path = (
#     "./apache/Apache_2k.log_templates.csv"
# )

loader = CSVLoader(file_path=file_path)
data = loader.load()

for record in data[:2]:
    print(record)

page_content='EventId: E1
EventTemplate: *** info [mice.c(<*>)]:' metadata={'source': './linux/Linux_2k.log_templates.csv', 'row': 0}
page_content='EventId: E2
EventTemplate: <*> HIGHMEM available.' metadata={'source': './linux/Linux_2k.log_templates.csv', 'row': 1}


In [10]:
data[0]

Document(metadata={'source': './linux/Linux_2k.log_templates.csv', 'row': 0}, page_content='EventId: E1\nEventTemplate: *** info [mice.c(<*>)]:')

In [11]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=0, separators=["\n\n", "\n", " ", ""])
all_splits = text_splitter.split_documents(data)

In [13]:
from langchain_community.vectorstores import FAISS 
from langchain_openai import OpenAIEmbeddings
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_community.embeddings import OllamaEmbeddings

# vectorstore = FAISS.from_documents(documents=all_splits, embedding=HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2"))

In [14]:
from langchain_ollama import OllamaEmbeddings

embeddings = OllamaEmbeddings(
    model="llama3.2",
)

In [15]:
vectorstore = FAISS.from_documents(all_splits, embeddings)

In [16]:
vectorstore.save_local("linux_vector_store")

In [13]:
query = '''
Retrieve Event Id from this error log
notice,workerEnv.init() ok /etc/httpd/conf/workers2.properties,E2,workerEnv.init() ok <*>
'''
docs = vectorstore.similarity_search_with_relevance_scores(query,fetch_k=4)
docs[0].page_content

'EventId: E3\nEventTemplate: mod_jk child workerEnv in error state <*>'

In [22]:
from langchain.chains import RetrievalQA
from langchain_openai import ChatOpenAI
from langchain_ollama import OllamaEmbeddings
from langchain_community.vectorstores import FAISS 

llm = ChatOpenAI(
    base_url = 'http://localhost:11434/v1',
    api_key='ollama', # required, but unused
    model="llama3.2"
)  

embeddings = OllamaEmbeddings(
    model="llama3.2",
)

vectorstore = FAISS.load_local("apache_vector_store", embeddings, allow_dangerous_deserialization=True)

input = "[Sun Dec 04 04:51:09 2005] [notice] jk2_init() Found child 6728 in scoreboard slot 6"

query = f'''
You are provided with Error Log that consists of "Content" and "Level".  
Retrieve Event Id and Event Template for this error log in list format with no textual data.

Input Format: [Time] [Level] [Content]

Provide the Output in the following format
Output Format: Event ID: [Event ID], Event Template: [Event Template]

Now provide the Output in the following format for the error 

{input}

'''

retriever = vectorstore.as_retriever(
    search_type="similarity",
    search_kwargs={"k": 1},
)
print(retriever)


qa = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=retriever,
    return_source_documents=True,      
) 
results = qa.invoke(query)
results

tags=['FAISS', 'OllamaEmbeddings'] vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x7db618096740> search_kwargs={'k': 1}


{'query': '\nYou are provided with Error Log that consists of "Content" and "Level".  \nRetrieve Event Id and Event Template for this error log in list format with no textual data.\n\nInput Format: [Time] [Level] [Content]\n\nProvide the Output in the following format\nOutput Format: Event ID: [Event ID], Event Template: [Event Template]\n\nNow provide the Output in the following format for the error \n\n[Sun Dec 04 04:51:09 2005] [notice] jk2_init() Found child 6728 in scoreboard slot 6\n\n',
 'result': 'Here is the output in the required format:\n\n* Event ID: E3\n* Event Template: mod_jk child workerEnv in error state \n\nNote that I could not extract any textual data from the input log as per your requirement. However, based on the provided event id (E3), I was able to match it with a possible template "mod_jk child workerEnv in error state".',
 'source_documents': [Document(metadata={'source': './apache/Apache_2k.log_templates.csv', 'row': 2}, page_content='EventId: E3\nEventTempl

In [15]:
results['result']

'Here is the output:\n\nOutput:\nEvent ID: E1, Event Template: jk2_init()\nEvent ID: E3, Event Template: mod_jk child workerEnv'

'Here is the output:\n\nOutput:\nEvent ID: E1, Event Template: jk2_init()\nEvent ID: E3, Event Template: mod_jk child workerEnv'

In [23]:
from langchain.chains import RetrievalQA
from langchain_openai import ChatOpenAI
from langchain_ollama import OllamaEmbeddings
from langchain_community.vectorstores import FAISS 

llm = ChatOpenAI(
    base_url = 'http://localhost:11434/v1',
    api_key='ollama', # required, but unused
    model="llama3.2"
)  

embeddings = OllamaEmbeddings(
    model="llama3.2",
)

vectorstore = FAISS.load_local("error_template_vector_store", embeddings, allow_dangerous_deserialization=True)

input = "[Sun Dec 04 04:51:09 2005] [notice] jk2_init() Found child 6728 in scoreboard slot 6"

query = f'''
You are provided with Error Log that consists of "Content" and "Level".  
Retrieve Event Id and Event Template for this error log in list format with no textual data.

Input Format: [Time] [Level] [Content]

Provide the Output in the following format
Output Format: Event ID: [Event ID], Event Template: [Event Template]

Now provide the Output in the following format for the error 

{input}

'''

retriever = vectorstore.as_retriever(
    search_type="similarity",
    search_kwargs={"k": 1},
)
print(retriever)

tags=['FAISS', 'OllamaEmbeddings'] vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x7db6180505e0> search_kwargs={'k': 1}


In [27]:
query = '''
Retrieve Event Id from this error log
notice,workerEnv.init() ok /etc/httpd/conf/workers2.properties,E2,workerEnv.init() ok <*>
'''
docs = vectorstore.similarity_search_with_relevance_scores(query,fetch_k=4)
docs[0]

/tmp/ipykernel_72904/1640382489.py:5: UserWarning: Relevance scores must be between 0 and 1, got [(Document(metadata={'source': './apache/Apache_2k.log_templates.csv', 'row': 2}, page_content='EventId: E3\nEventTemplate: mod_jk child workerEnv in error state <*>'), 0.164907401226967), (Document(metadata={'source': './apache/Apache_2k.log_templates.csv', 'row': 1}, page_content='EventId: E2\nEventTemplate: workerEnv.init() ok <*>'), 0.1078788376816644), (Document(metadata={'source': './apache/Apache_2k.log_templates.csv', 'row': 3}, page_content='EventId: E4\nEventTemplate: [client <*>] Directory index forbidden by rule: <*>'), -0.013616365234143446), (Document(metadata={'source': './apache/Apache_2k.log_templates.csv', 'row': 0}, page_content='EventId: E1\nEventTemplate: jk2_init() Found child <*> in scoreboard slot <*>'), -0.033750167648888496)]
  docs = vectorstore.similarity_search_with_relevance_scores(query,fetch_k=4)


(Document(metadata={'source': './apache/Apache_2k.log_templates.csv', 'row': 2}, page_content='EventId: E3\nEventTemplate: mod_jk child workerEnv in error state <*>'),
 0.164907401226967)

In [35]:
docs[0][0].page_content

'EventId: E3\nEventTemplate: mod_jk child workerEnv in error state <*>'